In [1]:
import pandas as pd
import os
import numpy as np
print(os.getcwd())


/srv/train_group3/Ying_Alex


In [106]:
def get_data(path, header_row=0):
    data = pd.read_csv(path, header=header_row)
    return data

In [107]:
def check_nan(df):
    # Assuming your data is a pandas DataFrame
    has_nan = df.isna().any(axis=1)  # Check for NaN values row-wise
    nan_indices = has_nan[has_nan].index  # Get the indices where NaN values exist

    if not nan_indices.empty:
        print("There are NaN values in the data.")
        print("Indices of NaN values:", nan_indices)
    else:
        print("There are no NaN values in the data.")

In [108]:
def check_resolution(df):
    #To identify which data in your DataFrame is at 6-hour resolution and which is at 1-hour resolution
    # Calculate the time differences between consecutive timestamps
    time_diffs = df.index.to_series().diff()
    print(time_diffs)

    # Identify the timestamps where the time difference is 6 hours (21600 seconds)
    six_hour_resolution_indices = time_diffs[time_diffs == pd.Timedelta(hours=6)].index

    # Print the indices where 6-hour resolution data is present
    print("Indices of 6-hour resolution data:")
    print(six_hour_resolution_indices)
    start_time = six_hour_resolution_indices.min()
    end_time = six_hour_resolution_indices.max()
    return start_time, end_time


In [109]:
def convert_resolution(start_time, end_time, data):
    # Create a new DataFrame with a DateTime index at 1-hour intervals
    one_hour_index = pd.date_range(start=start_time, end=end_time, freq='1H')
    one_hour_index.name = 'datetime'
    one_hour_data = pd.DataFrame(index=one_hour_index)

    # Use reindex to align the 6-hour resolution data with the 1-hour index
    resampled_data = data.reindex(one_hour_index)
    

    # Use linear interpolation to fill in the missing values
    for col in data.columns:
        resampled_data[col] = resampled_data[col].interpolate(method='linear')

    # Print the resulting DataFrame

    return resampled_data

In [110]:
def losses_data_preprocessing(data):
    #convert losses from kwh to mwh
    data['MWh']=data['kWh']/1000
    #convert quarter resolution to hourly resolution
    data['Zeitstempel'] = pd.to_datetime(data['Zeitstempel']) 
    data['time_start'] = data['Zeitstempel']-pd.Timedelta(minutes=15)
    data['datetime'] = data['time_start'].apply(lambda x: x.strftime('%Y-%m-%d %H:00:00'))
    data['datetime'] = pd.to_datetime(data['datetime']) 
    data_hour = data.groupby(['datetime'])['MWh'].sum().reset_index()
    data_hour.set_index('datetime', inplace=True)
    return data_hour
    
    

In [120]:
def temperature_data_preprocessing(data):
    data['datetime'] = pd.to_datetime(data['datetime'])
    data.set_index('datetime', inplace=True)
    check_nan(data)
    # get the time period for the data at 6 hour resolution
    start_time, end_time = check_resolution(temp)
    start_time = start_time - pd.Timedelta(hours=6)
    # resample the data at 6 hour resolution to 1 hour resolution
    resampled_data = convert_resolution(start_time, end_time, temp[start_time:end_time])
    start_time_for_rest = end_time + pd.Timedelta(hours=1)
    original_data = temp[start_time_for_rest:]
    # add first row for datetime 2019-01-01 00:00:00
    first_row_index = resampled_data.index[0] - pd.Timedelta(hours=1)
    first_row_data = resampled_data.iloc[0].values
    first_row = pd.DataFrame([first_row_data], columns=resampled_data.columns, index=[first_row_index])
    frist_part = pd.concat([first_row, resampled_data])
    print(frist_part.index)
    print(len(frist_part.index))
    print(len(frist_part.index.unique()))
    #concatenate all data together
    total = pd.concat([frist_part, original_data])

    # add row for datetime 2021-03-28 02:00:00
    row_index = pd.to_datetime('2021-03-28 02:00:00')
    row_before_index = row_index - pd.Timedelta(hours=1)
    row_before_data = total.loc[row_before_index].values
    row = pd.DataFrame([row_before_data], columns=resampled_data.columns, index=[row_index])
    total = pd.concat([total, row])
    print(total.index)
    print(len(total.index))
    print(len(total.index.unique()))
    # remove one entry for datetime 2021-10-31 02:00:00
    
    total = total[~total.index.duplicated(keep='first')]
    return total

In [154]:
def ntc_datapreprocessing(data, full_index):
    data['datetime'] = pd.to_datetime(data['datetime'])
    data.set_index('datetime', inplace=True)
    # Check for duplicate indices
#     duplicate_indices = data.index[data.index.duplicated()]
    data = fill_missing_data(data, full_index)
    data = data[~data.index.duplicated(keep='first')]
    return data
#     print(data.index)
#     print(len(data.index))
#     print(len(data.index.unique()))

In [159]:
ntc_path = '/home/jupyter-user_13/data/NTC.csv'
ntc = get_data(ntc_path)
print(len(ntc['datetime'].unique()))
full_index = losses.index
ntc = ntc_datapreprocessing(ntc, full_index)
print(ntc)

26301
2019-03-31 02:00:00
2019-03-31 01:00:00
[ 650. 4000. 1200. 2360. 1136. 1800. 3000. 1910.]
2020-03-29 02:00:00
2020-03-29 01:00:00
[1200. 4000. 1200. 1071. 1200.  800. 3000. 1910.]
2021-03-28 02:00:00
2021-03-28 01:00:00
[1200. 4000. 1400. 3560.  900.  800. 2385. 1910.]
                      CH_AT   CH_DE   CH_FR   CH_IT   AT_CH   DE_CH   FR_CH  \
2019-01-01 00:00:00   700.0  4000.0  1200.0  2513.0  1200.0   800.0  3000.0   
2019-01-01 01:00:00   700.0  4000.0  1200.0  2513.0  1200.0   800.0  3000.0   
2019-01-01 02:00:00   700.0  4000.0  1200.0  2513.0  1200.0   800.0  3000.0   
2019-01-01 03:00:00   700.0  4000.0  1200.0  2513.0  1200.0   800.0  3000.0   
2019-01-01 04:00:00   700.0  4000.0  1200.0  2513.0  1200.0   800.0  3000.0   
...                     ...     ...     ...     ...     ...     ...     ...   
2021-12-31 22:00:00  1200.0  4000.0  1400.0  3780.0  1200.0   800.0  3200.0   
2021-12-31 23:00:00  1200.0  4000.0  1400.0  2992.0  1200.0   800.0  3200.0   
2019-03-31 02

In [158]:
def fill_missing_data(df, full_index):
    for index in full_index:
        if index not in df.index:
            print(index)
            row_before_index = index - pd.Timedelta(hours=1)
            print(row_before_index)
            row_before_data = df.loc[row_before_index].values
            print(row_before_data)
            row = pd.DataFrame([row_before_data], columns=df.columns, index=[index])
            df = pd.concat([df, row])
    return df
            

In [112]:
def merge_data(df1, df2):
    print(len(df1.index.unique()))
    print(len(df2.index.unique()))
        # Check if the indices of df1 and df2 are exactly the same
    if df1.index.equals(df2.index):
        print("The indices are exactly the same.")
    else:
        print("The indices are not the same.")
    print(df1)
    print(df2)
    df_merge = df1.join(df2)
    return df_merge

In [160]:
temperature_path ='/home/jupyter-user_13/data/Forecast-temperature_new.csv'
temp = get_data(temperature_path)

temp = temperature_data_preprocessing(temp)


There are no NaN values in the data.
datetime
2019-01-01 01:00:00               NaT
2019-01-01 07:00:00   0 days 06:00:00
2019-01-01 13:00:00   0 days 06:00:00
2019-01-01 19:00:00   0 days 06:00:00
2019-01-02 01:00:00   0 days 06:00:00
                            ...      
2021-12-31 19:00:00   0 days 01:00:00
2021-12-31 20:00:00   0 days 01:00:00
2021-12-31 21:00:00   0 days 01:00:00
2021-12-31 22:00:00   0 days 01:00:00
2021-12-31 23:00:00   0 days 01:00:00
Name: datetime, Length: 11363, dtype: timedelta64[ns]
Indices of 6-hour resolution data:
DatetimeIndex(['2019-01-01 07:00:00', '2019-01-01 13:00:00',
               '2019-01-01 19:00:00', '2019-01-02 01:00:00',
               '2019-01-02 07:00:00', '2019-01-02 13:00:00',
               '2019-01-02 19:00:00', '2019-01-03 01:00:00',
               '2019-01-03 07:00:00', '2019-01-03 13:00:00',
               ...
               '2021-01-14 19:00:00', '2021-01-15 01:00:00',
               '2021-01-15 07:00:00', '2021-01-15 13:00:00',
 

In [128]:
losses_path = '/home/jupyter-user_13/data/Avtice-losses.csv'
losses = get_data(losses_path, 1)
losses = losses_data_preprocessing(losses)
print(losses)

                            MWh
datetime                       
2019-01-01 00:00:00  139.525004
2019-01-01 01:00:00  129.716036
2019-01-01 02:00:00  133.398074
2019-01-01 03:00:00  135.133852
2019-01-01 04:00:00  131.699424
...                         ...
2021-12-31 19:00:00  171.707318
2021-12-31 20:00:00  159.462903
2021-12-31 21:00:00  155.109520
2021-12-31 22:00:00  171.370277
2021-12-31 23:00:00  146.054791

[26304 rows x 1 columns]


In [134]:
for index in losses.index:
    if index not in ntc.index:
        print(index)

2019-03-31 02:00:00
2020-03-29 02:00:00
2021-03-28 02:00:00


In [59]:
print(type(losses.index[0]))

<class 'str'>


In [161]:
# merge losses and temperature
merged = merge_data(losses, temp)
merged = merge_data(merged, ntc)
print(merged)

26304
26304
The indices are not the same.
                            MWh
datetime                       
2019-01-01 00:00:00  139.525004
2019-01-01 01:00:00  129.716036
2019-01-01 02:00:00  133.398074
2019-01-01 03:00:00  135.133852
2019-01-01 04:00:00  131.699424
...                         ...
2021-12-31 19:00:00  171.707318
2021-12-31 20:00:00  159.462903
2021-12-31 21:00:00  155.109520
2021-12-31 22:00:00  171.370277
2021-12-31 23:00:00  146.054791

[26304 rows x 1 columns]
                     temperature_fore_ch  temperature_fore_fr  \
2019-01-01 00:00:00               4.1067             5.972900   
2019-01-01 01:00:00               4.1067             5.972900   
2019-01-01 02:00:00               3.7155             5.900717   
2019-01-01 03:00:00               3.3243             5.828533   
2019-01-01 04:00:00               2.9331             5.756350   
...                                  ...                  ...   
2021-12-31 20:00:00               8.0000             9.400000

In [17]:
merged

In [162]:
train_start_date ='2019-01-01 00'
train_end_date ='2021-09-30 23'
test_start_date ='2021-10-01 00'
test_end_date ='2021-12-31 23'


train_data = merged.loc[train_start_date : train_end_date]
test_data = merged.loc[test_start_date : test_end_date]



In [163]:
# prepare input data

def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data)-seq_length-24):
        sequences.append(data.iloc[i:i+seq_length])
        targets.append(data.iloc[(i+seq_length):(i+seq_length+24),0])

    sequence_array = np.array(sequences)
    targets_array = np.array(targets)
    return sequence_array, targets_array

In [102]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Step 1: Scale the input data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

seq_length = 24*7

X_train,y_train = create_sequences(train_data, seq_length)
X_test,y_test = create_sequences(test_data, seq_length)
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)

In [93]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Step 1: Scale the input data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

seq_length = 24*7

X_train,y_train = create_sequences(train_data, seq_length)
X_test,y_test = create_sequences(test_data, seq_length)
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.fit_transform(X_test)
scaler_y 

In [90]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(23904, 168, 5) (23904, 24) (2016, 168, 5) (2016, 24)


In [164]:
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)

In [165]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(23904, 840) (23904, 24) (2016, 840) (2016, 24)


In [71]:
# # Flatten the target data to make it compatible with the model's output
# y_train = y_train.reshape(-1, 24)
# y_test = y_test.reshape(-1, 24)

In [103]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(23904, 840) (23904, 24) (2016, 840) (2016, 24)


In [47]:
# x_train = x_train.reshape(x_train.shape[0], x_train.shape[1])
# y_train = y_train.reshape(y_train.shape[0], y_train.shape[1])
# x_test = x_test.reshape(x_test.shape[0], x_test.shape[1])
# y_test = y_test.reshape(y_test.shape[0], y_test.shape[1])

In [ ]:
import torch
import torch.nn as nn

class MyRegressionModel(nn.Module):
    def __init__(self):
        super(MyRegressionModel, self).__init__()

        # Define the layers of the neural network
        self.fc1 = nn.Linear(168 * 5, 128)  # Fully connected layer 1
        self.relu1 = nn.ReLU()              # ReLU activation
        self.fc2 = nn.Linear(128, 64)       # Fully connected layer 2
        self.relu2 = nn.ReLU()              # ReLU activation
        self.fc3 = nn.Linear(64, 24)        # Output layer (24 output neurons for regression)

    def forward(self, x):
        
        # Pass through the layers
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        
        return x
    def calculate_mae(self,y_true, y_pred):
        """
        Calculate the Mean Absolute Error (MAE) between true and predicted values.

        Args:
            y_true (torch.Tensor): True target values.
            y_pred (torch.Tensor): Predicted values.

        Returns:
            torch.Tensor: Calculated MAE.
        """
        absolute_errors = torch.abs(y_true - y_pred)
        mae = torch.mean(absolute_errors)
        return mae





#Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Convert data to PyTorch tensors and create datasets and dataloaders
# Convert data to PyTorch tensors
X_train_tensor = torch.Tensor(X_train).to(device)
y_train_tensor = torch.Tensor(y_train).to(device)
X_test_tensor = torch.Tensor(X_test).to(device)
y_test_tensor = torch.Tensor(y_test).to(device)

# Create DataLoader for training and testing data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Create an instance of the regression model
model = MyRegressionModel().to(device)

# Print the model architecture
# print(regression_model)
# Define the loss function (mean squared error) and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

# Training loop
num_epochs = 70
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Assuming 'model' is your PyTorch model
model_path = 'my_model.pth'  # Specify the path where you want to save the model

# Save the model to a file
torch.save(model.state_dict(), model_path)

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    # Calculate MAE during training (you can also do this for validation)
    mae = model.calculate_mae(y_test_tensor,y_pred)
    print(f"Epoch {epoch + 1}, Batch MAE: {mae.item()}")
#     mse = criterion(y_pred, y_test_tensor).item()
#     rmse = np.sqrt(mse)

# print(f"Root Mean Squared Error (RMSE): {rmse}")

Using device: cuda
Epoch 1/70, Loss: 765.8552499862916
Epoch 2/70, Loss: 600.1794930524367
Epoch 3/70, Loss: 568.0573679062135
Epoch 4/70, Loss: 557.9007893118629
Epoch 5/70, Loss: 544.4130713314934
Epoch 6/70, Loss: 543.5922348920037
Epoch 7/70, Loss: 534.4827181566208
Epoch 8/70, Loss: 539.1458074396306
Epoch 9/70, Loss: 530.3591009129815
Epoch 10/70, Loss: 519.3069537973658
Epoch 11/70, Loss: 519.4712660682393
Epoch 12/70, Loss: 519.6356202803831
Epoch 13/70, Loss: 520.2967575807622
Epoch 14/70, Loss: 513.8223208667123
Epoch 15/70, Loss: 509.61578287542824
Epoch 16/70, Loss: 504.5006361364681
Epoch 17/70, Loss: 505.15217027307193
Epoch 18/70, Loss: 501.4291784929082
Epoch 19/70, Loss: 494.5894559972426
Epoch 20/70, Loss: 498.583291201668
Epoch 21/70, Loss: 500.77223776751026
Epoch 22/70, Loss: 490.7286851036357
Epoch 23/70, Loss: 487.1821502032764
Epoch 24/70, Loss: 489.4169766023197
Epoch 25/70, Loss: 484.3865172850257
Epoch 26/70, Loss: 484.68082073028074
Epoch 27/70, Loss: 484.90

In [101]:
import torch
import torch.nn as nn

class MyRegressionModel(nn.Module):
    def __init__(self):
        super(MyRegressionModel, self).__init__()

        # Define the layers of the neural network
        self.fc1 = nn.Linear(168 * 5, 128)  # Fully connected layer 1
        self.relu1 = nn.ReLU()              # ReLU activation
        self.fc2 = nn.Linear(128, 64)       # Fully connected layer 2
        self.relu2 = nn.ReLU()              # ReLU activation
        self.fc3 = nn.Linear(64, 24)        # Output layer (24 output neurons for regression)

    def forward(self, x):
        
        # Pass through the layers
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        
        return x
    def calculate_mae(self,y_true, y_pred):
        """
        Calculate the Mean Absolute Error (MAE) between true and predicted values.

        Args:
            y_true (torch.Tensor): True target values.
            y_pred (torch.Tensor): Predicted values.

        Returns:
            torch.Tensor: Calculated MAE.
        """
        absolute_errors = np.abs(y_true - y_pred)
        mae = np.mean(absolute_errors)
        return mae





#Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Convert data to PyTorch tensors and create datasets and dataloaders
# Convert data to PyTorch tensors
X_train_tensor = torch.Tensor(X_train_scaled).to(device)
y_train_tensor = torch.Tensor(y_train).to(device)
X_test_tensor = torch.Tensor(X_test_scaled).to(device)
y_test_tensor = torch.Tensor(y_test).to(device)

# Create DataLoader for training and testing data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Create an instance of the regression model
model = MyRegressionModel().to(device)

# Print the model architecture
# print(regression_model)
# Define the loss function (mean squared error) and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

# Training loop
num_epochs = 70
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Assuming 'model' is your PyTorch model
model_path = 'my_model.pth'  # Specify the path where you want to save the model

# Save the model to a file
torch.save(model.state_dict(), model_path)

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_cpu = y_pred.cpu().detach().numpy()
    # Reverse the scaling for y_pred to the original scale
    scaler_y = MinMaxScaler()  # Create a new scaler for the output
    scaler_y.fit(y_pred_cpu)  # Fit the scaler on the original y data
    # Inverse transform the predictions to the original scale
    y_pred = scaler_y.inverse_transform(y_pred_cpu)
    # Calculate MAE during training (you can also do this for validation)
    mae = model.calculate_mae(y_pred, y_test_tensor.cpu().detach().numpy())
    print(f"Epoch {epoch + 1}, Batch MAE: {mae.item()}")
#     mse = criterion(y_pred, y_test_tensor).item()
#     rmse = np.sqrt(mse)

# print(f"Root Mean Squared Error (RMSE): {rmse}")

Using device: cuda
Epoch 1/70, Loss: 1326.283061797606
Epoch 2/70, Loss: 678.556767427985
Epoch 3/70, Loss: 655.5813101702196
Epoch 4/70, Loss: 642.3698813698509
Epoch 5/70, Loss: 636.6701553263129
Epoch 6/70, Loss: 624.4741025710489
Epoch 7/70, Loss: 625.4180011443276
Epoch 8/70, Loss: 620.6155681100121
Epoch 9/70, Loss: 614.0992363098471
Epoch 10/70, Loss: 616.3429489543731
Epoch 11/70, Loss: 610.4295145942566
Epoch 12/70, Loss: 603.8940141647257
Epoch 13/70, Loss: 595.0145363221194
Epoch 14/70, Loss: 587.4844712854069
Epoch 15/70, Loss: 576.1510704978903
Epoch 16/70, Loss: 572.9017796643915
Epoch 17/70, Loss: 562.4102448652135
Epoch 18/70, Loss: 563.2341297986036
Epoch 19/70, Loss: 559.579076348779
Epoch 20/70, Loss: 560.0663907464175
Epoch 21/70, Loss: 551.2220090978286
Epoch 22/70, Loss: 546.5443740273541
Epoch 23/70, Loss: 542.5317083348565
Epoch 24/70, Loss: 539.4178721382019
Epoch 25/70, Loss: 538.8653987129742
Epoch 26/70, Loss: 538.4528893455465
Epoch 27/70, Loss: 530.4951283

In [44]:
# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [78]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch.optim as optim
# Define the layers of the neural network
        self.fc1 = nn.Linear(168 * 5, 128)  # Fully connected layer 1
        self.relu1 = nn.ReLU()              # ReLU activation
        self.fc2 = nn.Linear(128, 64)       # Fully connected layer 2
        self.relu2 = nn.ReLU()              # ReLU activation
        self.fc3 = nn.Linear(64, 24)        # Output layer (24 output neurons for regression)



MyRegressionModel(
  (conv1): Conv2d(5, 16, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=5120, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=24, bias=True)
)


In [151]:
#Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Convert data to PyTorch tensors and create datasets and dataloaders
# Convert data to PyTorch tensors
X_train_tensor = torch.Tensor(X_train).to(device)
y_train_tensor = torch.Tensor(y_train).to(device)
X_test_tensor = torch.Tensor(X_test).to(device)
y_test_tensor = torch.Tensor(y_test).to(device)

# Create DataLoader for training and testing data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)



# # Define the regression model
# class RegressionModel(nn.Module):
#     def __init__(self):
#         super(RegressionModel, self).__init__()
#         self.lstm = nn.LSTM(input_size=168, hidden_size=128, num_layers=2, batch_first=True)
#         self.fc = nn.Linear(128, 24)  # 24 is the output dimension

#     def forward(self, x):
#         out, _ = self.lstm(x)
#         out = self.fc(out[:, -1, :])  # Only take the output at the last time step
#         return out
    


#Create an instance of the model
# model = RegressionModel().to(device)
# model = RegressionModel()

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # Input size to 128 hidden units
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)  # 128 hidden units to 64 hidden units
        self.fc3 = nn.Linear(64, 24)  # 64 hidden units to 24 output units (output dimension)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x.squeeze(dim=1)  # Remove the middle dimension to match target shape

input_dim = X_train.shape[2] 
model = NeuralNetwork(input_dim).to(device)
# Define the loss function (mean squared error) and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    mse = criterion(y_pred, y_test_tensor).item()
    rmse = np.sqrt(mse)

print(f"Root Mean Squared Error (RMSE): {rmse}")

Using device: cuda
Epoch 1/50, Loss: 949.4913760098544
Epoch 2/50, Loss: 616.8321649887982
Epoch 3/50, Loss: 591.0670209262461
Epoch 4/50, Loss: 571.5438948850581
Epoch 5/50, Loss: 557.6853449204389
Epoch 6/50, Loss: 551.3929916626629
Epoch 7/50, Loss: 553.5069704106785
Epoch 8/50, Loss: 547.8167818446848
Epoch 9/50, Loss: 545.3272486396014
Epoch 10/50, Loss: 544.7291768119934
Epoch 11/50, Loss: 545.6561182195491
Epoch 12/50, Loss: 541.4460505521233
Epoch 13/50, Loss: 542.3478499631831
Epoch 14/50, Loss: 541.9111430122254
Epoch 15/50, Loss: 539.285353308693
Epoch 16/50, Loss: 536.9506991789302
Epoch 17/50, Loss: 543.0647250088779
Epoch 18/50, Loss: 536.5910766927954
Epoch 19/50, Loss: 537.4339408670518
Epoch 20/50, Loss: 538.2079525707877
Epoch 21/50, Loss: 536.3120889918689
Epoch 22/50, Loss: 531.6399360697537
Epoch 23/50, Loss: 532.0558044107203
Epoch 24/50, Loss: 534.7366685510319
Epoch 25/50, Loss: 534.703904503807
Epoch 26/50, Loss: 532.2325707909895
Epoch 27/50, Loss: 531.0279855